In [147]:
import numpy as np
import torch

In [148]:
# set logger and enforce reproducibility
from GPErks.log.logger import get_logger
from GPErks.utils.random import set_seed
log = get_logger()
seed = 8
set_seed(seed)  # reproducible sampling

**2D function example**: Currin et al. (1988)

$f(x) = \left[1 - \exp{\left(1 - \dfrac{1}{2 x_2}\right)}\right]\,\left(\dfrac{2300 x_{1}^3 + 1900 x_{1}^2 + 2092 x_{1} + 60}{100 x_{1}^3 + 500 x_{1}^2 + 4 x_{1} + 20}\right)$

In [149]:
# function to learn (normally a high-dimensional, expensive deterministic model)
from GPErks.utils.test_functions import currin_exp
f = lambda X: np.array([currin_exp(x) for x in X])
D = 2

In [150]:
# build dataset
from GPErks.gp.data.dataset import Dataset
dataset = Dataset.build_from_function(
    f,
    D,
    n_train_samples=20,
    n_test_samples=25,
    design="lhs",
    seed=seed,
)

In [151]:
# choose likelihood
from gpytorch.likelihoods import GaussianLikelihood
likelihood = GaussianLikelihood()

In [152]:
# choose mean function
from gpytorch.means import LinearMean
mean_function = LinearMean(input_size=dataset.input_size)

In [153]:
# choose kernel
from gpytorch.kernels import RBFKernel, ScaleKernel
kernel = ScaleKernel(RBFKernel(ard_num_dims=dataset.input_size))

In [154]:
# choose metrics
from torchmetrics import MeanSquaredError, R2Score
metrics = [MeanSquaredError(), R2Score()]

In [155]:
# define experiment
from GPErks.gp.experiment import GPExperiment
experiment = GPExperiment(
    dataset,
    likelihood,
    mean_function,
    kernel,
    n_restarts=3,
    metrics=metrics,
    seed=seed  # reproducible training
)

In [156]:
# dump experiment in config file
import os
notebook_full_path = os.path.join(os.getcwd(), nb_name)
file = os.path.basename(notebook_full_path)
config_file = file.replace(".ipynb", ".ini")
experiment.save_to_config_file(config_file)

In [157]:
# load experiment from config file
from GPErks.gp.experiment import load_experiment_from_config_file
experiment = load_experiment_from_config_file(
    config_file,
    dataset
)

In [158]:
# choose training options: device + optimizer
device = "cuda" if torch.cuda.is_available() else "cpu"
optimizer = torch.optim.Adam(experiment.model.parameters(), lr=0.1)

In [159]:
# train model
from GPErks.train.emulator import GPEmulator
emulator = GPEmulator(experiment_2, device)

In [160]:
# snapshotting
from GPErks.serialization.path import posix_path
from GPErks.train.snapshot import EveryEpochSnapshottingCriterion

snapshot_dir = posix_path(".", "snapshot", "pippo")
train_restart_template = "restart_{restart}"
train_epoch_template = "epoch_{epoch}.pth"

snpc = EveryEpochSnapshottingCriterion(
    posix_path(
        snapshot_dir,
        train_restart_template,
    ),
    train_epoch_template
)

# training
emulator.train(optimizer, snapshotting_criterion=snpc)

INFO:2021-08-03 19:15:25,623:emulator:train:L83: Training emulator...
INFO:2021-08-03 19:15:25,625:emulator:train:L99: Running restart 1...
/home/sl18/Documents/Python/GPErks/venv/lib/python3.8/site-packages/gpytorch/models/exact_gp.py:282: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(
INFO:2021-08-03 19:15:25,643:emulator:_train_once:L308: [  1/100] Training Loss: 0.8316 - MeanSquaredError: 0.0899 - R2Score: 0.9101
INFO:2021-08-03 19:15:25,650:emulator:_train_once:L308: [  2/100] Training Loss: 0.8316 - MeanSquaredError: 0.0899 - R2Score: 0.9101
INFO:2021-08-03 19:15:25,657:emulator:_train_once:L308: [  3/100] Training Loss: 0.8316 - MeanSquaredError: 0.0899 - R2Score: 0.9101
INFO:2021-08-03 19:15:25,664:emulator:_train_once:L308: [  4/100] Training Loss: 0.8316 - MeanSquaredError: 0.0899 - R2Score: 0.9101
INFO:2021-08-03 19:15:25,675:emulator:_train_once:L308: [  5/100] Training Loss: 0.8316 - MeanSquaredError: 0.08

INFO:2021-08-03 19:15:26,049:emulator:_train_once:L308: [ 60/100] Training Loss: 0.8316 - MeanSquaredError: 0.0899 - R2Score: 0.9101
INFO:2021-08-03 19:15:26,054:emulator:_train_once:L308: [ 61/100] Training Loss: 0.8316 - MeanSquaredError: 0.0899 - R2Score: 0.9101
INFO:2021-08-03 19:15:26,062:emulator:_train_once:L308: [ 62/100] Training Loss: 0.8316 - MeanSquaredError: 0.0899 - R2Score: 0.9101
INFO:2021-08-03 19:15:26,071:emulator:_train_once:L308: [ 63/100] Training Loss: 0.8316 - MeanSquaredError: 0.0899 - R2Score: 0.9101
INFO:2021-08-03 19:15:26,079:emulator:_train_once:L308: [ 64/100] Training Loss: 0.8316 - MeanSquaredError: 0.0899 - R2Score: 0.9101
INFO:2021-08-03 19:15:26,086:emulator:_train_once:L308: [ 65/100] Training Loss: 0.8316 - MeanSquaredError: 0.0899 - R2Score: 0.9101
INFO:2021-08-03 19:15:26,092:emulator:_train_once:L308: [ 66/100] Training Loss: 0.8316 - MeanSquaredError: 0.0899 - R2Score: 0.9101
INFO:2021-08-03 19:15:26,100:emulator:_train_once:L308: [ 67/100] Tra

INFO:2021-08-03 19:15:26,465:emulator:_train_once:L308: [ 21/100] Training Loss: 0.5174 - MeanSquaredError: 0.0439 - R2Score: 0.9561
INFO:2021-08-03 19:15:26,475:emulator:_train_once:L308: [ 22/100] Training Loss: 0.5174 - MeanSquaredError: 0.0439 - R2Score: 0.9561
INFO:2021-08-03 19:15:26,485:emulator:_train_once:L308: [ 23/100] Training Loss: 0.5174 - MeanSquaredError: 0.0439 - R2Score: 0.9561
INFO:2021-08-03 19:15:26,492:emulator:_train_once:L308: [ 24/100] Training Loss: 0.5174 - MeanSquaredError: 0.0439 - R2Score: 0.9561
INFO:2021-08-03 19:15:26,500:emulator:_train_once:L308: [ 25/100] Training Loss: 0.5174 - MeanSquaredError: 0.0439 - R2Score: 0.9561
INFO:2021-08-03 19:15:26,507:emulator:_train_once:L308: [ 26/100] Training Loss: 0.5174 - MeanSquaredError: 0.0439 - R2Score: 0.9561
INFO:2021-08-03 19:15:26,514:emulator:_train_once:L308: [ 27/100] Training Loss: 0.5174 - MeanSquaredError: 0.0439 - R2Score: 0.9561
INFO:2021-08-03 19:15:26,521:emulator:_train_once:L308: [ 28/100] Tra

INFO:2021-08-03 19:15:26,856:emulator:_train_once:L308: [ 83/100] Training Loss: 0.5174 - MeanSquaredError: 0.0439 - R2Score: 0.9561
INFO:2021-08-03 19:15:26,863:emulator:_train_once:L308: [ 84/100] Training Loss: 0.5174 - MeanSquaredError: 0.0439 - R2Score: 0.9561
INFO:2021-08-03 19:15:26,870:emulator:_train_once:L308: [ 85/100] Training Loss: 0.5174 - MeanSquaredError: 0.0439 - R2Score: 0.9561
INFO:2021-08-03 19:15:26,875:emulator:_train_once:L308: [ 86/100] Training Loss: 0.5174 - MeanSquaredError: 0.0439 - R2Score: 0.9561
INFO:2021-08-03 19:15:26,881:emulator:_train_once:L308: [ 87/100] Training Loss: 0.5174 - MeanSquaredError: 0.0439 - R2Score: 0.9561
INFO:2021-08-03 19:15:26,888:emulator:_train_once:L308: [ 88/100] Training Loss: 0.5174 - MeanSquaredError: 0.0439 - R2Score: 0.9561
INFO:2021-08-03 19:15:26,895:emulator:_train_once:L308: [ 89/100] Training Loss: 0.5174 - MeanSquaredError: 0.0439 - R2Score: 0.9561
INFO:2021-08-03 19:15:26,902:emulator:_train_once:L308: [ 90/100] Tra

INFO:2021-08-03 19:15:27,225:emulator:_train_once:L308: [ 44/100] Training Loss: 1.3645 - MeanSquaredError: 0.0722 - R2Score: 0.9278
INFO:2021-08-03 19:15:27,231:emulator:_train_once:L308: [ 45/100] Training Loss: 1.3645 - MeanSquaredError: 0.0722 - R2Score: 0.9278
INFO:2021-08-03 19:15:27,236:emulator:_train_once:L308: [ 46/100] Training Loss: 1.3645 - MeanSquaredError: 0.0722 - R2Score: 0.9278
INFO:2021-08-03 19:15:27,241:emulator:_train_once:L308: [ 47/100] Training Loss: 1.3645 - MeanSquaredError: 0.0722 - R2Score: 0.9278
INFO:2021-08-03 19:15:27,247:emulator:_train_once:L308: [ 48/100] Training Loss: 1.3645 - MeanSquaredError: 0.0722 - R2Score: 0.9278
INFO:2021-08-03 19:15:27,252:emulator:_train_once:L308: [ 49/100] Training Loss: 1.3645 - MeanSquaredError: 0.0722 - R2Score: 0.9278
INFO:2021-08-03 19:15:27,257:emulator:_train_once:L308: [ 50/100] Training Loss: 1.3645 - MeanSquaredError: 0.0722 - R2Score: 0.9278
INFO:2021-08-03 19:15:27,265:emulator:_train_once:L308: [ 51/100] Tra


Bias: 0.5027
Weights: tensor([ 0.2713, -1.2729])
Outputscale: 1.9854
Lengthscales: tensor([0.7254, 1.0798])
Likelihood noise: 0.1015


(OrderedDict([('likelihood.noise_covar.raw_noise', tensor([-2.2377])),
              ('mean_module.weights',
               tensor([[ 0.2713],
                       [-1.2729]])),
              ('mean_module.bias', tensor([0.5027])),
              ('covar_module.raw_outputscale', tensor(1.8377)),
              ('covar_module.base_kernel.raw_lengthscale',
               tensor([[0.0636, 0.6647]]))]),
 <GPErks.train.train_stats.TrainStats at 0x7fcb739e6d30>)

In [161]:
# inference on stored test set
from GPErks.perks.inference import Inference
inference = Inference(emulator)
inference.summary()

                   Score
MeanSquaredError  1.1552
R2Score           0.8428


In [162]:
# loading emulator
from pathlib import Path
# best_model_file = posix_path(".",
#     Path(snpc.snapshot_dir).parent.as_posix(),
#     "best_model.pth"
# )  # for some reason, symbolic link not working on a notebook

best_restart = 1
best_epoch = 100
best_model_file = snpc.get_snapshot_file_path(best_restart, best_epoch)

emulator_2 = GPEmulator(experiment_2, device)
emulator_2.model = torch.load(best_model_file, map_location=torch.device(device))

In [163]:
inference = Inference(emulator)
inference.summary()

                   Score
MeanSquaredError  1.1552
R2Score           0.8428
